Chest X-ray (Pneumonia): Classification Model Using Transfer Learning

LIST OF IMPORTS

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Dropout,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix

DATA PREPARATION

In [2]:
# The paths to the train, validation, and test directories
train_path = "dechets/Train"
val_path = "dechets/dev"
test_path = "dechets/test"

# The input image size
input_size = (224, 224)

# The batch size for the data generators
batch_size = 32

In [3]:
# The data generators for the training, validation, and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    )
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=input_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_path, target_size=input_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=input_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

Found 850 images belonging to 3 classes.


Found 225 images belonging to 3 classes.
Found 224 images belonging to 3 classes.


In [4]:
train_generator.class_indices

{'bois': 0, 'construction': 1, 'metal': 2}

In [5]:
# The number of classes in the dataset
num_classes = 3

USING VGG

In [6]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential

In [7]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))
for layer in base_model.layers:
    layer.trainable = False


In [8]:
# Create a new model that includes the VGG16 layers and adds a few new layers on top
model_mobilenet = Sequential([
    base_model,
    #Flatten(),
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),

    #Dropout(0.4),
    Dense(3, activation='softmax')
])

In [9]:
model_mobilenet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 3,492,035
Trainable params: 263,171
Non-trainable params: 3,228,864
_________________________________________________________________


In [10]:
# Compile the model
model_mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
model_mobilenet.fit(train_generator, steps_per_epoch=len(train_generator),batch_size=32, epochs=8, validation_data=val_generator, validation_steps=len(val_generator))

Epoch 1/8
27/27 [==============================] - 17s 580ms/step - loss: 0.6394 - accuracy: 0.7400 - val_loss: 0.4065 - val_accuracy: 0.8400
Epoch 2/8
27/27 [==============================] - 15s 560ms/step - loss: 0.2586 - accuracy: 0.9000 - val_loss: 0.5413 - val_accuracy: 0.8044
Epoch 3/8
27/27 [==============================] - 17s 639ms/step - loss: 0.1705 - accuracy: 0.9424 - val_loss: 0.4053 - val_accuracy: 0.8400
Epoch 4/8
27/27 [==============================] - 17s 646ms/step - loss: 0.1271 - accuracy: 0.9494 - val_loss: 0.4578 - val_accuracy: 0.8222
Epoch 5/8
27/27 [==============================] - 17s 642ms/step - loss: 0.0685 - accuracy: 0.9835 - val_loss: 0.3912 - val_accuracy: 0.8622
Epoch 6/8
27/27 [==============================] - 17s 642ms/step - loss: 0.0434 - accuracy: 0.9918 - val_loss: 0.4588 - val_accuracy: 0.8356
Epoch 7/8
27/27 [==============================] - 17s 646ms/step - loss: 0.0320 - accuracy: 0.9918 - val_loss: 0.4609 - val_accuracy: 0.8444
Epoch 

In [12]:
# Evaluate the model on the test set
test_loss_VGG, test_acc_VGG = model_mobilenet.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', test_loss_VGG)
print('Test accuracy:', test_acc_VGG)

7/7 [==============================] - 4s 490ms/step - loss: 0.4698 - accuracy: 0.8214
Test loss: 0.469824880361557
Test accuracy: 0.8214285969734192


In [13]:
# Make predictions on the test set
test_generator.reset()
preds = model_mobilenet.predict(test_generator, steps=len(test_generator), verbose=1)
y_true = test_generator.classes
y_pred = np.argmax(preds, axis=1)

7/7 [==============================] - 4s 473ms/step


In [14]:
#Print the classification report and confusion matrix
print('Classification Report:')
print(classification_report(y_true, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.88      0.81        74
           1       0.82      0.85      0.84        75
           2       0.92      0.73      0.81        75

    accuracy                           0.82       224
   macro avg       0.83      0.82      0.82       224
weighted avg       0.83      0.82      0.82       224

Confusion Matrix:
[[65  7  2]
 [ 8 64  3]
 [13  7 55]]


In [19]:
model_mobilenet.save("classifier_saved_model")

INFO:tensorflow:Assets written to: classifier_saved_model\assets


INFO:tensorflow:Assets written to: classifier_saved_model\assets


In [23]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("classifier_saved_model")
tflite_model = converter.convert()
open("classifier.tflite", "wb").write(tflite_model)


13857264